In [1]:
!pip install -q datasets
!pip install -q evaluate
!pip install -q seqeval
!pip install -q -U transformers
!pip install -q transformers[torch]
!pip install -q -U accelerate

In [2]:
model_name = "KoichiYasuoka/bert-base-russian-upos"  # Name of the BERT model
max_len = 512  # Maximum length of input sequences for the model

In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Initialize tokenizer with specified model name and maximum length
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=max_len)

tokenizer_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [20]:
from datasets import load_dataset

# Load dataset from "iluvvatar/RuNNE"
dataset = load_dataset("iluvvatar/RuNNE")
dataset

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for iluvvatar/RuNNE contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/iluvvatar/RuNNE
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'entities'],
        num_rows: 461
    })
    test: Dataset({
        features: ['id', 'text', 'entities'],
        num_rows: 93
    })
    dev: Dataset({
        features: ['id', 'text', 'entities'],
        num_rows: 323
    })
})

In [21]:
entities = set()  # Initialize set to store unique entity labels

# Extract unique entity labels from the 'entities' field in the train split
for sent in dataset['train']['entities']:
    entities = entities.union(set(ent.split()[2] for ent in sent))

# Initialize mappings for entity labels to IDs and vice versa
entity2id = {'O': 0}
id2entity = {0: 'O'}

# Assign IDs to each entity label (BIO format)
for idx, entity in enumerate(entities):
    entity2id[f'B-{entity}'] = 2 * idx + 1
    id2entity[2 * idx + 1] = f'B-{entity}'
    entity2id[f'I-{entity}'] = 2 * idx + 2
    id2entity[2 * idx + 2] = f'I-{entity}'

print("Entity to ID mapping:")
print(entity2id)
print("\nID to Entity mapping:")
print(id2entity)

Entity to ID mapping:
{'O': 0, 'B-IDEOLOGY': 1, 'I-IDEOLOGY': 2, 'B-NUMBER': 3, 'I-NUMBER': 4, 'B-ORGANIZATION': 5, 'I-ORGANIZATION': 6, 'B-TIME': 7, 'I-TIME': 8, 'B-LAW': 9, 'I-LAW': 10, 'B-FAMILY': 11, 'I-FAMILY': 12, 'B-CRIME': 13, 'I-CRIME': 14, 'B-ORDINAL': 15, 'I-ORDINAL': 16, 'B-WORK_OF_ART': 17, 'I-WORK_OF_ART': 18, 'B-DATE': 19, 'I-DATE': 20, 'B-PENALTY': 21, 'I-PENALTY': 22, 'B-AGE': 23, 'I-AGE': 24, 'B-CITY': 25, 'I-CITY': 26, 'B-PRODUCT': 27, 'I-PRODUCT': 28, 'B-COUNTRY': 29, 'I-COUNTRY': 30, 'B-LANGUAGE': 31, 'I-LANGUAGE': 32, 'B-MONEY': 33, 'I-MONEY': 34, 'B-STATE_OR_PROVINCE': 35, 'I-STATE_OR_PROVINCE': 36, 'B-RELIGION': 37, 'I-RELIGION': 38, 'B-EVENT': 39, 'I-EVENT': 40, 'B-DISTRICT': 41, 'I-DISTRICT': 42, 'B-PROFESSION': 43, 'I-PROFESSION': 44, 'B-PERSON': 45, 'I-PERSON': 46, 'B-LOCATION': 47, 'I-LOCATION': 48, 'B-FACILITY': 49, 'I-FACILITY': 50, 'B-AWARD': 51, 'I-AWARD': 52, 'B-PERCENT': 53, 'I-PERCENT': 54, 'B-DISEASE': 55, 'I-DISEASE': 56, 'B-NATIONALITY': 57, 'I-NA

In [22]:
import pandas as pd
from datasets import Dataset, concatenate_datasets


def dataset_maker(file_name='/kaggle/input/nlp-a3-data/train.jsonl', is_train=True):
    """
    Function to create a dataset from a JSON file.

    Args:
        file_name (str, optional): Path to the JSON file.
        is_train (bool, optional): Dataset is for training or not.

    Returns:
        Dataset: Dataset created from the JSON file.
    """
    df = pd.read_json(file_name, lines=True)    
    if is_train:
        df = df.rename({'sentences': 'text', 'ners': 'entities'}, axis=1)[['text', 'entities']]
        entities = []
        for row in df.iterrows():
            entities.append([f'{ent[0]} {ent[1]} {ent[2]}' for ent in row[1]['entities']])
        df['entities'] = entities
    else:
        df = df.rename({'senences': 'text'}, axis=1)
    dataset = Dataset.from_pandas(df, preserve_index=False)
    return dataset

json_dataset = dataset_maker()
json_dataset

Dataset({
    features: ['text', 'entities'],
    num_rows: 519
})

In [23]:
def format_hf(example):
    """
    Function to format entity annotations for Hugging Face datasets.

    Args:
        example (dict): Example containing 'entities' field.

    Returns:
        dict: Example with formatted entity annotations.
    """
    for j in range(len(example['entities'])):
        s, e, E = example['entities'][j].split()
        example['entities'][j] = f'{s} {int(e) - 1} {E}'  # Format end position by subtracting 1
    return example


dataset['train'] = dataset['train'].map(format_hf)
dataset['test'] = dataset['test'].map(format_hf)

0 12 PERSON


Map:   0%|          | 0/93 [00:00<?, ? examples/s]

0 11 PERSON


In [24]:
dataset['train'] = concatenate_datasets([json_dataset, dataset['train']])
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'entities', 'id'],
        num_rows: 980
    })
    test: Dataset({
        features: ['id', 'text', 'entities'],
        num_rows: 93
    })
    dev: Dataset({
        features: ['id', 'text', 'entities'],
        num_rows: 323
    })
})

In [25]:
def mapper(example):
    """
    Function to map text and entity annotations to tokenized inputs and NER tags.

    Args:
        example (dict): Example containing 'text' and 'entities' fields.

    Returns:
        dict: Tokenized inputs with NER tags.
    """
    text = example['text']  # Extract text from example
    entities = example['entities']  # Extract entity annotations
    ner_tags = [0] * max_len  # Initialize NER tags with zeros

    # Iterate over entity annotations and assign BIO tags to token positions
    for entity_str in entities:
        start, end, entity = entity_str.split()
        start, end = int(start), int(end)
        token_start = len(tokenizer.tokenize(text[0:start]))  # Start token index
        subtext = tokenizer.tokenize(text[start:end + 1])  # Tokenize subtext containing entity
        for token_id in range(token_start, min(len(ner_tags), token_start + len(subtext) + 1)):
            if token_id == token_start:
                if ner_tags[token_id] == 0:
                    ner_tags[token_id] = entity2id[f'B-{entity}']  # Assign B-tag for beginning of entity
            else:
                if ner_tags[token_id] == 0:
                    ner_tags[token_id] = entity2id[f'I-{entity}']  # Assign I-tag for inside of entity

    # Tokenize text and assign NER tags to corresponding token positions
    result = tokenizer(text, max_length=max_len, padding='max_length', truncation=True)
    result['labels'] = ner_tags

    # Check if the length of labels and input_ids doesn't match
    assert len(result['labels']) != len(result['input_ids'])
    return result

In [26]:
train_dataset = dataset['train'].map(mapper)
test_dataset = dataset['test'].map(mapper)

Map:   0%|          | 0/980 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1521 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/93 [00:00<?, ? examples/s]

In [27]:
import numpy as np
import evaluate

seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    """
    Function to compute evaluation metrics for NER predictions.

    Args:
        p (tuple): Tuple containing predictions and labels.

    Returns:
        dict: Dictionary containing precision, recall, F1-score, and accuracy.
    """
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Extract true predictions and true labels excluding padding tokens
    true_predictions = [
        [id2entity[p] for (p, l) in zip(prediction, label) if l != 29]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2entity[l] for (p, l) in zip(prediction, label) if l != 29]
        for prediction, label in zip(predictions, labels)
    ]

    # Compute evaluation metrics using seqeval
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    
    # Return metrics as a dictionary
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

2024-04-26 10:37:28.447839: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 10:37:28.447938: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 10:37:28.566078: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [28]:
model = AutoModelForTokenClassification.from_pretrained(model_name, 
                                                        num_labels=len(id2entity), 
                                                        id2label=id2entity, 
                                                        label2id=entity2id, 
                                                        ignore_mismatched_sizes=True)

config.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/709M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForTokenClassification were not initialized from the model checkpoint at KoichiYasuoka/bert-base-russian-upos and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([89, 768]) in the checkpoint and torch.Size([59, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([89]) in the checkpoint and torch.Size([59]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="bert",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=5,
    load_best_model_at_end=True,
    report_to='tensorboard',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.504085,0.412660,0.501594,0.452801,0.878696
2,No log,0.314675,0.534152,0.674814,0.596300,0.910888
3,No log,0.255738,0.648750,0.700319,0.673549,0.932405
4,No log,0.256172,0.662317,0.722848,0.691260,0.935639
5,No log,0.265385,0.660642,0.730287,0.693721,0.934983
6,No log,0.283204,0.661091,0.739214,0.697973,0.934497
7,No log,0.291517,0.670979,0.738576,0.703157,0.936019
8,No log,0.299882,0.671050,0.737513,0.702714,0.934941
9,0.219100,0.302813,0.678509,0.742827,0.709213,0.936717
10,0.219100,0.306222,0.679672,0.740489,0.708778,0.936970


/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

TrainOutput(global_step=620, training_loss=0.18186494727288524, metrics={'train_runtime': 610.8658, 'train_samples_per_second': 16.043, 'train_steps_per_second': 1.015, 'total_flos': 2562027834777600.0, 'train_loss': 0.18186494727288524, 'epoch': 10.0})

In [31]:
from transformers import pipeline

# Check the inference
text = 'Умер создатель первого в мире индексного фонда Джон Богл Основатель и бывший глава самой крупной инвестиционной компании мира The Vanguard Group Джон Богл умер в возрасте 89 лет в среду, 16 января 2019 года, в своём доме в городе Брин-Мар (Пенсильвания, США). На момент смерти финансиста активы, находящиеся под управлением основанной им Vanguard Group превышают 5 трлн долларов США. При этом состояние самого Богла оцениваются менее 100 млн долларов. Инвестор гордился тем, что не стал миллиардером. Джон Богл в 1975 году основал компанию The Vanguard Group, где создал первый в мире общедоступный взаимный индексный фонд Vanguard 500. Он всю жизнь убеждал всех, что расходы на комиссию управляющих компаний не оправдывают себя — все они зарабатывают в среднем не больше, чем при пассивных инвестициях, но при этом комиссия управляющих заметно понижает прибыль инвесторов. «Выбирайте фонды с низкими издержками и осторожно относитесь к дополнительным расходам», — писал инвестиционный управляющий в одном из своих бестселлеров. Родившийся 8 мая 1929 года Богл скончался от онкологического заболевания. Он пережил шесть инфарктов начиная с 1960 года. В 1996 году ему пересадили сердце. Тем не менее, он возглавлял The Vanguard Group до 1999 года, после чего руководил Bogle Financial Markets Resource Center, где занимался анализом процессов, влияющих на взаимные фонды, финансовые рынки и инвесторов. Богл написал 12 книг, которые разошлись по миру тиражом в 1,1 млн экземпляров. В 2004 году журнал Time назвал его одним из 100 самых влиятельных людей мира.'
classifier = pipeline('ner', 
                      tokenizer=tokenizer, 
                      model=AutoModelForTokenClassification.from_pretrained('/kaggle/working/bert/checkpoint-558', 
                                                                            local_files_only=True), 
                      aggregation_strategy='simple')
res = classifier(text)
res[:10]

[{'entity_group': 'EVENT',
  'score': 0.99567187,
  'word': 'Умер',
  'start': 0,
  'end': 4},
 {'entity_group': 'ORDINAL',
  'score': 0.98959434,
  'word': 'создатель первого',
  'start': 5,
  'end': 22},
 {'entity_group': 'PERSON',
  'score': 0.99872506,
  'word': 'фонда Джон Богл',
  'start': 41,
  'end': 56},
 {'entity_group': 'PROFESSION',
  'score': 0.9943047,
  'word': 'бывший глава',
  'start': 70,
  'end': 82},
 {'entity_group': 'PROFESSION',
  'score': 0.8594332,
  'word': 'крупной инвестиционной компании',
  'start': 89,
  'end': 120},
 {'entity_group': 'ORGANIZATION',
  'score': 0.9783225,
  'word': 'мира The Vanguard Group',
  'start': 121,
  'end': 144},
 {'entity_group': 'PERSON',
  'score': 0.9989994,
  'word': 'Джон Богл',
  'start': 145,
  'end': 154},
 {'entity_group': 'EVENT',
  'score': 0.99530494,
  'word': 'умер',
  'start': 155,
  'end': 159},
 {'entity_group': 'AGE',
  'score': 0.99162006,
  'word': 'возрасте 89 лет',
  'start': 162,
  'end': 177},
 {'entity_gr

In [36]:
start_chars = ' .(),'
end_chars = ' .(,'

def format_inference(inference):
    """
    Function to format inference results by adjusting start and end positions of tokens.

    Args:
        inference (list): List of tokens with start, end, and entity_group fields.

    Returns:
        list: Formatted list of token spans with adjusted start and end positions.
    """
    result = []
    for token in inference:
        start, end = token['start'], token['end']
        # Adjust start position by skipping characters in start_chars
        for c in token['word']:
            if c in start_chars:
                start += 1
            else:
                break
        # Adjust end position by skipping characters in end_chars
        for c in token['word'][::-1]:
            if c in end_chars:
                end -= 1
            else:
                break
        # If start position is greater than end position, skip the token
        if start > end:
            continue
        result += [[start, end - 1, token['entity_group']]]
    return result

# Check
for e in format_inference(res)[:25]:
    print(f'"{text[e[0]: e[1] + 1]}"', "|", e[2], e[0], e[1])

"Умер" | EVENT 0 3
"создатель первого" | ORDINAL 5 21
"фонда Джон Богл" | PERSON 41 55
"бывший глава" | PROFESSION 70 81
"крупной инвестиционной компании" | PROFESSION 89 119
"мира The Vanguard Group" | ORGANIZATION 121 143
"Джон Богл" | PERSON 145 153
"умер" | EVENT 155 158
"возрасте 89 лет" | AGE 162 176
"в" | DATE 178 178
"среду, 16" | TIME 180 188
"января" | DATE 190 195
"2019 года" | TIME 197 205
"городе Брин-Мар" | CITY 223 237
"енсильвания" | STATE_OR_PROVINCE 241 251
"США" | COUNTRY 254 256
"смерти финансиста" | PROFESSION 270 286
"им Vanguard Group" | ORGANIZATION 335 351
"превышают 5 трлн долларов США" | MONEY 353 381
"самого Богла" | PERSON 403 414
"оцениваются менее 100 млн долларов" | MONEY 416 449
"стал миллиардером" | PROFESSION 482 498
"Джон Богл" | PERSON 501 509
"в 1975 году" | DATE 511 521
"компанию The Vanguard Group" | ORGANIZATION 531 557


In [37]:
def get_answer(text, id):
    """
    Function to get NER predictions for a given text.

    Args:
        text (str): Input text to extract entities from.
        id (str): Identifier for the text.

    Returns:
        dict: Dictionary containing text identifier and extracted entities.
    """
    res = []  # Initialize list to store NER predictions
    # Process text in chunks of 512 characters and obtain NER predictions
    for i in range(0, len(text), 512):
        offset = len(res)
        res += classifier(text[i:min(len(text),i+512)])  # Get NER predictions for chunk
        # Adjust start and end positions of predictions based on chunk offset
        for j in range(offset, len(res)):
            res[j]['start'] += i
            res[j]['end'] += i
    # Format NER predictions
    formatted_res = format_inference(res)
    return {'id': id, 'ners': formatted_res}

In [38]:
text = 'Умер создатель первого в мире индексного фонда Джон Богл Основатель и бывший глава самой крупной инвестиционной компании мира The Vanguard Group Джон Богл умер в возрасте 89 лет в среду, 16 января 2019 года, в своём доме в городе Брин-Мар (Пенсильвания, США). На момент смерти финансиста активы, находящиеся под управлением основанной им Vanguard Group превышают 5 трлн долларов США. При этом состояние самого Богла оцениваются менее 100 млн долларов. Инвестор гордился тем, что не стал миллиардером. Джон Богл в 1975 году основал компанию The Vanguard Group, где создал первый в мире общедоступный взаимный индексный фонд Vanguard 500. Он всю жизнь убеждал всех, что расходы на комиссию управляющих компаний не оправдывают себя — все они зарабатывают в среднем не больше, чем при пассивных инвестициях, но при этом комиссия управляющих заметно понижает прибыль инвесторов. «Выбирайте фонды с низкими издержками и осторожно относитесь к дополнительным расходам», — писал инвестиционный управляющий в одном из своих бестселлеров. Родившийся 8 мая 1929 года Богл скончался от онкологического заболевания. Он пережил шесть инфарктов начиная с 1960 года. В 1996 году ему пересадили сердце. Тем не менее, он возглавлял The Vanguard Group до 1999 года, после чего руководил Bogle Financial Markets Resource Center, где занимался анализом процессов, влияющих на взаимные фонды, финансовые рынки и инвесторов. Богл написал 12 книг, которые разошлись по миру тиражом в 1,1 млн экземпляров. В 2004 году журнал Time назвал его одним из 100 самых влиятельных людей мира.'
print(len(text))
get_answer(text, 0)

1558


{'id': 0,
 'ners': [[0, 3, 'EVENT'],
  [5, 21, 'ORDINAL'],
  [41, 55, 'PERSON'],
  [70, 81, 'PROFESSION'],
  [112, 119, 'PROFESSION'],
  [121, 143, 'ORGANIZATION'],
  [145, 153, 'PERSON'],
  [155, 158, 'EVENT'],
  [162, 176, 'AGE'],
  [178, 205, 'DATE'],
  [223, 237, 'CITY'],
  [241, 251, 'STATE_OR_PROVINCE'],
  [254, 256, 'COUNTRY'],
  [270, 286, 'PROFESSION'],
  [335, 351, 'ORGANIZATION'],
  [353, 381, 'MONEY'],
  [403, 414, 'PERSON'],
  [416, 449, 'MONEY'],
  [452, 459, 'PROFESSION'],
  [482, 498, 'PROFESSION'],
  [501, 509, 'PERSON'],
  [513, 521, 'DATE'],
  [523, 529, 'EVENT'],
  [531, 557, 'ORGANIZATION'],
  [564, 576, 'ORDINAL'],
  [618, 621, 'PRODUCT'],
  [623, 630, 'ORGANIZATION'],
  [632, 634, 'PRODUCT'],
  [965, 996, 'PROFESSION'],
  [1032, 1054, 'DATE'],
  [1056, 1059, 'PERSON'],
  [1061, 1069, 'EVENT'],
  [1071, 1100, 'DISEASE'],
  [1106, 1118, 'NUMBER'],
  [1120, 1126, 'DISEASE'],
  [1127, 1128, 'EVENT'],
  [1130, 1148, 'DATE'],
  [1151, 1161, 'DATE'],
  [1163, 1183, 'EVE

In [39]:
dev_dataset = dataset_maker('/kaggle/input/nlp-a3-data/test.jsonl', is_train=False)
dev_dataset

Dataset({
    features: ['text', 'id'],
    num_rows: 65
})

In [46]:
dev_answers = []
for sample in dev_dataset:
    dev_answers += [get_answer(sample['text'], sample['id'])]

In [48]:
import json

def save_jsonl(data, filename):
    with open(filename, 'w') as file:
        for item in data:
            json.dump(item, file)
            file.write('\n')

save_jsonl(dev_answers, 'test.jsonl')